In [1]:
import scanpy as sc
import numpy as np
import scgen as scg
import pandas as pd
import sys
sys.path.append("/work/users/mh823zote/projects/cov/integration/analysis")
import helper_VAE as hVAE
import scipy.sparse
import scgen
import matplotlib.pyplot as plt
import seaborn as sns
import harmonypy as hpy
import anndata
import random

import os

/home/sc.uni-leipzig.de/mh823zote/miniconda3/envs/scgen/lib/python3.9/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


# Read in

In [2]:
species = 'ma'
celltype = 'Non_Classical_Monocytes'
model_save_string = os.getcwd()
model_save_string = model_save_string + '/' + celltype + '/H0_01_model_' + celltype + '.pt'
model = scgen.SCGEN.load(model_save_string)

INFO     Using data from adata.X                                                             
INFO     Computing library size prior per batch                                              
INFO     Registered keys:['X', 'batch_indices', 'local_l_mean', 'local_l_var', 'labels']     
INFO     Successfully registered anndata object containing 356 cells, 1059 vars, 5 batches, 1
         labels, and 0 proteins. Also registered 0 extra categorical covariates and 0 extra  
         continuous covariates.                                                              


/home/sc.uni-leipzig.de/mh823zote/miniconda3/envs/scgen/lib/python3.9/site-packages/scvi/data/_anndata.py:301: UserWarning: Training will be faster when sparse matrix is formatted as CSR. It is safe to cast before model initialization.
  warnings.warn(


In [4]:
adata = model.adata
lat_adata = hVAE.get_latent_representation_object(model,adata)

#hVAE.prepare_umap(lat_adata)

#sc.pl.umap(lat_adata,color = 'timepoint')

D0 = hVAE.filter_adata_obs(lat_adata,col_name='timepoint',val='D0')
hd_D2 = hVAE.filter_adata_obs(lat_adata,col_name='timepoint',val='hd_D2')
hd_D3 = hVAE.filter_adata_obs(lat_adata,col_name='timepoint',val='hd_D3')
ld_D2 = hVAE.filter_adata_obs(lat_adata,col_name='timepoint',val='ld_D2')
ld_D2 = hVAE.filter_adata_obs(lat_adata,col_name='timepoint',val='ld_D2')

lat_adata = hVAE.merge_adata(hVAE.merge_adata(hVAE.merge_adata(hVAE.merge_adata(ld_D2,hd_D2),ld_D2),hd_D3),D0)

delta = hVAE.get_delta_in_latent_space(D0,hd_D2)

/home/sc.uni-leipzig.de/mh823zote/miniconda3/envs/scgen/lib/python3.9/site-packages/anndata/_core/anndata.py:1755: FutureWarning: The AnnData.concatenate method is deprecated in favour of the anndata.concat function. Please use anndata.concat instead.

See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  warnings.warn(
/home/sc.uni-leipzig.de/mh823zote/miniconda3/envs/scgen/lib/python3.9/site-packages/anndata/_core/anndata.py:1755: FutureWarning: The AnnData.concatenate method is deprecated in favour of the anndata.concat function. Please use anndata.concat instead.

See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  warnings.warn(
/home/sc.uni-leipzig.de/mh823zote/miniconda3/envs/scgen/lib/python3.9/site-packages/anndata/_core/anndata.py:1755: FutureWarning: The AnnData.concatenate method is deprecated in favour of the anndata.concat function. Please use anndata.concat instead.

See the tutorial for co

In [5]:
test_set = sc.read_h5ad(celltype + '/H0_01_test_Non_Classical_Monocytes_.h5ad')
lat_adata_test = hVAE.get_latent_representation_object(model,test_set)

lat_D0_test = lat_adata_test[lat_adata_test.obs['timepoint'] == 'D0']

lat_D0_test_shifted = hVAE.shift_adata_in_latent_space(lat_D0_test,delta)

lat_D0_test_shifted.obs['timepoint'] = 'hd_D2 predicted'

lat_test_predicted = hVAE.merge_adata(lat_adata_test,lat_D0_test_shifted)

INFO     Input adata not setup with scvi. attempting to transfer anndata setup               
INFO     Using data from adata.X                                                             
INFO     Computing library size prior per batch                                              
INFO     Registered keys:['X', 'batch_indices', 'local_l_mean', 'local_l_var', 'labels']     
INFO     Successfully registered anndata object containing 258 cells, 1059 vars, 5 batches, 1
         labels, and 0 proteins. Also registered 0 extra categorical covariates and 0 extra  
         continuous covariates.                                                              


/home/sc.uni-leipzig.de/mh823zote/miniconda3/envs/scgen/lib/python3.9/site-packages/scvi/data/_anndata.py:301: UserWarning: Training will be faster when sparse matrix is formatted as CSR. It is safe to cast before model initialization.
  warnings.warn(
/home/sc.uni-leipzig.de/mh823zote/miniconda3/envs/scgen/lib/python3.9/site-packages/anndata/_core/anndata.py:1755: FutureWarning: The AnnData.concatenate method is deprecated in favour of the anndata.concat function. Please use anndata.concat instead.

See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  warnings.warn(


In [6]:
decoded_test = hVAE.decode_latent_object(model,lat_test_predicted,test_set)

In [16]:
decoded_test.obs

,uniform_name_overview,timepoint,hamster,set,_scvi_batch,_scvi_labels,_scvi_local_l_mean,_scvi_local_l_var,batch
rn,,,,,,,,,
pr_D0_Z1_B_AAACGCTGTCCACTTC-1-0,Non_Classical_Monocytes,D0,Ha1,test,0,0,6.093823,0.032374,0
pr_D0_Z1_B_AATCACGTCCTACGGG-1-0,Non_Classical_Monocytes,D0,Ha1,test,0,0,6.093823,0.032374,0
pr_D0_Z1_B_ACATTTCAGCGAGTAC-1-0,Non_Classical_Monocytes,D0,Ha1,test,0,0,6.093823,0.032374,0
pr_D0_Z1_B_AGGGCTCCACAAGGTG-1-0,Non_Classical_Monocytes,D0,Ha1,test,0,0,6.093823,0.032374,0
pr_D0_Z1_B_AGGTCATAGGTCACAG-1-0,Non_Classical_Monocytes,D0,Ha1,test,0,0,6.093823,0.032374,0
...,...,...,...,...,...,...,...,...,...
pr_D0_Z1_B_TCGAACAGTGGCGCTT-1-1,Non_Classical_Monocytes,hd_D2 predicted,Ha1,test,0,0,6.093823,0.032374,1
pr_D0_Z1_B_TCGCAGGAGCACTAGG-1-1,Non_Classical_Monocytes,hd_D2 predicted,Ha1,test,0,0,6.093823,0.032374,1
pr_D0_Z1_B_TGCATCCCAAGAGTAT-1-1,Non_Classical_Monocytes,hd_D2 predicted,Ha1,test,0,0,6.093823,0.032374,1


In [17]:
gene_expression_df = pd.DataFrame(decoded_test.X,columns = decoded_test.var.index,index = decoded_test.obs.index)

obs_df = decoded_test.obs[['timepoint','uniform_name_overview']]

obs_df.to_csv(celltype + '/HO_03_meta_' + celltype + '_' + species + '.csv')

gene_expression_df.to_csv(celltype + '/HO_03_gene_expression_' + celltype + '_' + species + '.csv')